<a href="https://colab.research.google.com/github/Ash100/PPI/blob/main/FromComp_Protein_Protein_Docking_with_MEGADOCK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Protein Protein Docking with MEGADOCK**
Hello,
I am **Dr. Ashfaq Ahmad**, working in the field of Structure Biology and Bioinformatics. You can watch a video tutorial on https://www.youtube.com/@Bioinformaticsinsights

 This notebook is managed from the published work of *Masahito Ohue*. If you are planning to publish results generated from this noteboook, please cite

**1. MEGADOCK-on-Colab: an easy-to-use protein–protein docking tool on Google Colaboratory**

**2. MEGADOCK 4.0: an ultra–high-performance protein–protein docking software for heterogeneous supercomputers**.

You are not allowed to use this program commercially, in case you are planning to use it commercially, please take a prior permission from Ohue lab.

##Upload Your Protein Structures for Computer
Once you click, an upload tab will appear

In [ ]:
from google.colab import files
#@markdown **Receptor PDB** (`-R`)
print("upload receptor PDB file (.pdb)")
rup = files.upload()
rfilename = list(rup.keys())[0]

In [ ]:
#@markdown **Ligand PDB** (`-L`)
print("upload ligand PDB files (.pdb)")
lup = files.upload()
lfilename = list(lup.keys())[0]

In [12]:
#@markdown MEGADOCK parameters (if you want to change)
t = "6" #@param {type:"string"}
#D = 0 #@param {type:"string"}
N = "10800" #@param {type:"string"}
outfile_name = "dock.out" #@param {type:"string"}

In [ ]:
#@title Install all the Required software

# MEGADOCK
!git clone https://github.com/akiyamalab/MEGADOCK
!git clone https://github.com/NVIDIA/cuda-samples
!apt-get install -y libfftw3-dev libfftw3-single3

%cd ./MEGADOCK
!make -j 2 -f Makefile.colab

# biopython
!pip -q install biopython

# NGLView
!pip install nglview==3.0.8

!jupyter-nbextension enable nglview --py --sys-prefix

In [ ]:
#@title MEGADOCK parameter setting
import os
os.environ['MDt'] = str(t)
os.environ['MDN'] = str(N)
os.environ['MDPDBR'] = rfilename
os.environ['MDPDBL'] = lfilename
os.environ['MDOF'] = str(outfile_name)

!bash -c "mv /content/$MDPDBR ."
!bash -c "mv /content/$MDPDBL ."

In [ ]:
#@title Run MEGADOCK
!./megadock-gpu -R $MDPDBR -L $MDPDBL -t $MDt -N $MDN -o $MDOF

The Below script will allow you to visualize docking result of the Best Pose. If you are unable to see, that is fine. We will download the result and can analyze in offline mode using PyMol, Discovery Visualizer, VMD, Chimera or any other.

In [7]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os
import sys
from urllib.request import urlretrieve
import Bio
from Bio import PDB
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
import pylab
import urllib
import pandas as pd
import nglview as nv
from collections import Counter
from Bio.PDB import PDBParser,MMCIFParser

In [ ]:
#@title Show the 1st solution in PDB viewer (NGLView)
!./decoygen lig.1.pdb $MDPDBL $MDOF 1
!cat $MDPDBR lig.1.pdb | sed 's/END//g' > complex1.pdb
from google.colab import output
output.enable_custom_widget_manager()
import nglview as nv
view = nv.show_structure_file("complex1.pdb")
view

In [ ]:
#@title Show the top 5 solutions in PDB viewer (NGLView)
!./decoygen lig.2.pdb $MDPDBL $MDOF 2
!./decoygen lig.3.pdb $MDPDBL $MDOF 3
!./decoygen lig.4.pdb $MDPDBL $MDOF 4
!./decoygen lig.5.pdb $MDPDBL $MDOF 5
!cat complex1.pdb lig.2.pdb lig.3.pdb lig.4.pdb lig.5.pdb | sed 's/END//g' > complex5s.pdb
from google.colab import output
output.enable_custom_widget_manager()
import nglview as nv
view = nv.show_structure_file("complex5s.pdb")
view

In [ ]:
#@title PPI score calculation (for protein-protein interaction prediction)
!./ppiscore $MDOF $MDN

##Understand the PPIscore generate above in terms of Interaction
The value of the PPI Score (the value of $E$ in the above cell) can predict whether two protein chains will interact or not.  
The approximate values of precision (positive predictive value, PPV) and PPI Score are shown in the Figure below.  
* The precision is about 10% when $E > 8$ is predicted as "PPI positive."
* The precision is about 50% when $E > 10$ is predicted as "PPI positive."
* The precision is about 80% when $E > 12$ is predicted as "PPI positive."

![](http://drive.google.com/uc?export=view&id=1aVl9yRh-E4HXtn6AQ1fjrMb0M7Nx-wxH)
(from Ohue M, _et al_. _Jikkenigaku_, 37(9):1469, 2019.)